In [4]:
import os
import re
import json
import PyPDF2
import docx
from openai import OpenAI

from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    print("API Key carregada com sucesso!")
else:
    print("⚠️ API Key não foi encontrada!")

API Key carregada com sucesso!


In [ ]:
client = OpenAI(api_key=api_key)

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text() or ""
    return text

def extract_text_from_docx(docx_path):
    doc = docx.Document(docx_path)
    return "\n".join([p.text for p in doc.paragraphs])

def extract_text(file_path):
    ext = os.path.splitext(file_path)[1].lower()
    if ext == ".pdf":
        return extract_text_from_pdf(file_path)
    elif ext == ".docx":
        return extract_text_from_docx(file_path)
    else:
        return None

system_prompt = (
    "Você é um assistente que extrai dados estruturados de TCCs. "
    "Retorne um JSON válido com os seguintes campos: Autor, Título, Ano de publicação, Local de publicação, "
    "Orientador(a), Coorientador(a), Resumo, Palavras-chave, Introdução, Conclusão. "
    "As seções Resumo, Introdução e Conclusão devem conter os textos completos, sem alterações. Deve conter todos os parágrafos."
    "Não adicione comentários ou explicações."
)

# Pasta com os TCCs
pasta_tccs = "TCCs"
arquivos_tcc = [f for f in os.listdir(pasta_tccs) if f.lower().endswith(('.pdf', '.docx'))]

for arquivo in arquivos_tcc:
    caminho_arquivo = os.path.join(pasta_tccs, arquivo)
    print(f"📄 Processando: {arquivo}")

    texto_tcc = extract_text(caminho_arquivo)
    if not texto_tcc:
        print(f"⚠️ Não foi possível extrair texto de {arquivo}. Pulando...")
        continue

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"Texto do TCC:\n{texto_tcc}"}
            ]
        )

        resposta = response.choices[0].message.content
        match = re.search(r"\{(?:[^{}]|())*\}", resposta, re.DOTALL)
        
        if match:
            json_str = match.group()
            try:
                dados_extraidos = json.loads(json_str)
                os.makedirs("resultados_json", exist_ok=True)
                nome_saida = f"{os.path.splitext(arquivo)[0]}_dados.json"
                caminho_saida = os.path.join("resultados_json", nome_saida)

                with open(caminho_saida, "w", encoding="utf-8") as f:
                    json.dump(dados_extraidos, f, ensure_ascii=False, indent=4)

                print(f"✅ Extraído com sucesso: {nome_saida}")
            except json.JSONDecodeError:
                print(f"❌ JSON inválido para {arquivo}.")
                print(json_str)
        else:
            print(f"❌ Nenhum JSON encontrado na resposta de {arquivo}.")
            print(resposta)
    
    except Exception as e:
        print(f"❌ Erro ao processar {arquivo}: {e}")
